<a href="https://colab.research.google.com/github/zosiak00/uczenie_maszynowe_ZK/blob/case3/case3_ZK_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Stwórz model generujący sygnały kupna i sprzedaży na rynku

Decyzje ma generować jeden z algorytmów uczenia maszynowego, po optymalizacji hiperparametrów

Zrób backtest wybranego modelu, użyj biblioteki przystosowanej do backtestów

Pamiętaj o wnioskach i wizualizacji wyników


In [1]:
pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=6070392c43e550e8b9637e85fa0212321a3475c108f440c3a30d258299328197
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.6 MB/s eta 0:00:00


In [3]:
pip install yfinance

In [6]:
!pip3 install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=c5e98830628e84253268963502fa6be6093ac03d3040480f4086526968e79147
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [7]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime
from typing import Callable
import pandas_datareader.data as web
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import backtrader as bt
import ta
from backtesting import Strategy, Backtest
from sklearn.metrics import balanced_accuracy_score

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [9]:
# odczyt danych
train_data = yf.download('AAPL', start='2021-01-01', end='2023-12-31')
test_data = yf.download('AAPL', start='2024-01-01', end='2024-05-06')
train_data = train_data.dropna()
test_data = test_data.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
# wskaźniki
train_data_ind = train_data
train_data_ind['SMA20'] = ta.trend.sma_indicator(train_data['Close'], window=20)
train_data_ind['SMA50'] = ta.trend.sma_indicator(train_data['Close'], window=50)
train_data_ind['RSI'] = ta.momentum.rsi(train_data['Close'], window=14)
train_data_ind['MACD'] = ta.trend.macd_diff(train_data['Close'])
train_data_ind['StochK'] = ta.momentum.stoch(train_data['High'], train_data['Low'], train_data['Close'], window=14, smooth_window=3)
train_data_ind['StochD'] = ta.momentum.stoch_signal(train_data['High'], train_data['Low'], train_data['Close'], window=14, smooth_window=3)
train_data_ind['CCI'] = ta.trend.cci(train_data['High'], train_data['Low'], train_data['Close'], window=20)
train_data_ind['ATR'] = ta.volatility.average_true_range(train_data['High'], train_data['Low'], train_data['Close'], window=14)
train_data_ind['MFI'] = ta.volume.money_flow_index(train_data['High'], train_data['Low'], train_data['Close'], train_data['Volume'], window=14)

test_data['SMA20'] = ta.trend.sma_indicator(test_data['Close'], window=20)
test_data['SMA50'] = ta.trend.sma_indicator(test_data['Close'], window=50)
test_data['RSI'] = ta.momentum.rsi(test_data['Close'], window=14)
test_data['MACD'] = ta.trend.macd_diff(test_data['Close'])
test_data['StochK'] = ta.momentum.stoch(test_data['High'], test_data['Low'], test_data['Close'], window=14, smooth_window=3)
test_data['StochD'] = ta.momentum.stoch_signal(test_data['High'], test_data['Low'], test_data['Close'], window=14, smooth_window=3)
test_data['CCI'] = ta.trend.cci(test_data['High'], test_data['Low'], test_data['Close'], window=20)
test_data['ATR'] = ta.volatility.average_true_range(test_data['High'], test_data['Low'], test_data['Close'], window=14)
test_data['MFI'] = ta.volume.money_flow_index(test_data['High'], test_data['Low'], test_data['Close'], test_data['Volume'], window=14)

train_data = train_data.dropna()
test_data = test_data.dropna()

train_data['Signal'] = np.where(train_data['Close'].shift(-1) > train_data['Close'], 1, 0)
test_data['Signal'] = np.where(test_data['Close'].shift(-1) > test_data['Close'], 1, 0)

X_train = train_data[['SMA20', 'SMA50', 'RSI', 'MACD', 'StochK', 'StochD', 'CCI', 'ATR', 'MFI']]
y_train = train_data['Signal']
X_test = test_data[['SMA20', 'SMA50', 'RSI', 'MACD', 'StochK', 'StochD', 'CCI', 'ATR', 'MFI']]
y_test = test_data['Signal']

In [24]:
param_grid = {
     'n_estimators': [100, 200, 300],
     'learning_rate': [0.1, 0.01, 0.05],
     'colsample_bytree': [0.5, 0.8, 1.0],
     'max_depth': range(3, 9, 1)
}

In [25]:
from xgboost import XGBClassifier
model = XGBClassifier()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.5, 0.8, 1.0],
                         'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(3, 9),
                         'n_estimators': [100, 200, 300]})

In [26]:
# backtesting

class MLStrategy_ind(Strategy):
    def init(self):
        self.sma20 = self.I(ta.trend.sma_indicator, pd.Series(self.data.Close), window=20)
        self.sma50 = self.I(ta.trend.sma_indicator, pd.Series(self.data.Close), window=50)
        self.rsi = self.I(ta.momentum.rsi, pd.Series(self.data.Close), window=14)
        self.macd = self.I(ta.trend.macd_diff, pd.Series(self.data.Close))
        self.stochk = self.I(ta.momentum.stoch, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.stochd = self.I(ta.momentum.stoch_signal, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.cci = self.I(ta.trend.cci, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=20)
        self.atr = self.I(ta.volatility.average_true_range, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), window=14)
        self.mfi = self.I(ta.volume.money_flow_index, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), pd.Series(self.data.Volume), window=14)

    def next(self):
        features = [
            self.sma20[-1], self.sma50[-1],
            self.rsi[-1], self.macd[-1],
            self.stochk[-1], self.stochd[-1],
            self.cci[-1], self.atr[-1],
            self.mfi[-1]
        ]

        signal = model.predict([features])[0]

        if signal == 1 and not self.position:
            self.buy()
        elif signal == 0 and self.position:
            self.sell()

In [27]:
best_model = grid_search.best_estimator_

model = XGBClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [28]:
bt = Backtest(test_data, MLStrategy_ind, cash=100000, commission=0.002)
stats = bt.run()
bt.plot()
plt.show()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [29]:
print(stats)

Start                     2024-03-13 00:00:00
End                       2024-05-03 00:00:00
Duration                     51 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]                7.158301
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [31]:
trades = stats['_trades']
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
